In [ ]:
import numpy as np
import os
dtype = np.float32

In [ ]:
import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

In [ ]:
import json
import os
path = "/home/jgryu/Weight_compression/Wparam_dataset/path_json"
list = os.listdir(path)

for l in list :
    file_path = os.path.join(path, l)
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    for i in range(len(data)):
        data[i] = data[i].replace("/home/jgryu/Weight_compression", "")
        
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# V3
각 tensor의 path를 train val split해서 json 파일로 저장

In [ ]:
import glob
import random
import json
def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

save_path = '/home/jgryu/Weight_compression/model_param_dataset/'
dataset_path = '/home/jgryu/Weight_compression/model_param_tensor/'

tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)

print(f'all : {len(tenosr_path_list)}')

data = {'attn':[], 'mlp':[], 'embed':[], 'layernorm':[]}   

random.seed(100)

for weight_type in ['attn', 'mlp', 'embed', 'layernorm']: 
    for path in tenosr_path_list:
        if contains_all_substrings(path.lower(), [weight_type]):
            data[weight_type].append(path)
    print(f'{weight_type} : {len(data[weight_type])}')


    random.shuffle(data[weight_type])

    split_index = int(0.8 * len(data[weight_type]))
    train_data = data[weight_type][:split_index]
    validation_data = data[weight_type][split_index:]

    file_path_train = os.path.join(save_path, f'{weight_type}_tensor_path_train.json')
    file_path_validation = file_path_train.replace('train', 'val')

    with open(file_path_train, 'w') as json_file:
        json.dump(train_data, json_file, indent=4)
    with open(file_path_validation, 'w') as json_file:
        json.dump(validation_data, json_file, indent=4)

# V2
state_dict tensor별로 저장

In [ ]:
import numpy as np
import os
dtype = np.float32

import torch
import torchvision
import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

def get_ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

model_zoo_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_zoo/huggingface'
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
# model_filter = ['-13b', '-small']
model_filter = ['meta-llama-3-8b', 'meta-llama-3.1-8b']

model_list = os.listdir(model_zoo_path)
print()
ckpt_path_list = []
for ck in model_list:
    if check_contains_any_substrings(ck.lower(), model_filter):
        ckpt_path = get_ckpt_path(os.path.join(model_zoo_path, ck))
        if ckpt_path is not None:
            ckpt_path_list.append(ckpt_path)

print(len(ckpt_path_list))

save_dir = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor'

for ckpt_path in tqdm.tqdm(ckpt_path_list):
    model_name = ckpt_path.split('/')[-3]
    print(model_name)
    model_name = model_name.split('--')
    
    save_path = os.path.join(save_dir, model_name[1], model_name[2])

    if os.path.isdir(save_path) and bool(os.listdir(save_path)):
        print('### Skip ###')
        continue
    try:
        model = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True, trust_remote_code=True)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        state_dict = model.state_dict()
        for k, v in state_dict.items():
            np.save(os.path.join(save_path, k.replace(".", "-")), v)
            
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {ckpt_path}")
        continue



4
models--meta-llama--Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]


models--meta-llama--Meta-Llama-3-8B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.20s/it]


models--meta-llama--Meta-Llama-3.1-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.08s/it]


models--meta-llama--Meta-Llama-3-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.54s/it]


In [3]:
len(model_list)

201

In [ ]:
def get_ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

In [ ]:
model_zoo_path = '/home/jgryu/Weight_compression/model_zoo/huggingface'
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
model_list = os.listdir(model_zoo_path)
ckpt_path_list = []
for ck in model_list:
    if check_contains_any_substrings(ck.lower(), model_filter):
        ckpt_path = get_ckpt_path(os.path.join(model_zoo_path, ck))
        if ckpt_path is not None:
            ckpt_path_list.append(ckpt_path)

In [ ]:
save_dir = '/home/jgryu/Weight_compression/model_param_tensor'

for ckpt_path in ckpt_path_list:
    try:
        model = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {ckpt_path}")
        continue
    
    model_name = ckpt_path.split('/')[-3]
    print(model_name)
    model_name = model_name.split('--')
    
    save_path = os.path.join(save_dir, model_name[1], model_name[2])
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    if os.path.isdir(save_path) and not os.listdir(save_path):
        continue

    state_dict = model.state_dict()
    for k, v in state_dict.items():
        np.save(os.path.join(save_path, k.replace(".", "-")), v)